# 03_data_load.ipynb

Neste notebook vamos carregar o arquivo Parquet processado (`dados_projetos_limpos.parquet`) e persistir os dados em um banco PostgreSQL.

Antes de executar, certifique-se de que as variáveis de conexão estão disponíveis no ambiente (por exemplo via `.env`):

- PG_HOST
- PG_PORT
- PG_DB
- PG_USER
- PG_PASSWORD

E de ter levantado o container com o banco de dados postegrees usando o comando:

docker run --name meu-postgres -e POSTGRES_PASSWORD=minha_senha_secreta -e POSTGRES_USER=meu_usuario -e POSTGRES_DB=meu_banco -p 5432:5432 -d postgres

Também verifique se as dependências estão instaladas: `sqlalchemy`, `psycopg2-binary` e `pandas`.

In [6]:
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Carrega .env
load_dotenv()

ROOT = Path(os.getcwd()).resolve().parents[0]
PROCESSED_PATH = ROOT / 'data' / 'processed'
print('Parquet file:', PROCESSED_PATH)

# Lê o parquet
print("\nDataFrame de Projetos:")
df_projetos = pd.read_parquet(str(PROCESSED_PATH) + '/dados_projetos_limpos.parquet')
print('Loaded rows:', len(df_projetos))
print(df_projetos.dtypes)

print("\nDataFrame de Executores:")
df_executores = pd.read_parquet(str(PROCESSED_PATH) + '/dados_executores_limpos.parquet')
print('Loaded rows:', len(df_executores))
print(df_executores.dtypes)

print("\nDataFrame de Eixos:")
df_eixos = pd.read_parquet(str(PROCESSED_PATH) + '/dados_eixos_limpos.parquet')
print('Loaded rows:', len(df_eixos))
print(df_eixos.dtypes)

print("\nDataFrame de Subtipos:")
df_subtipos = pd.read_parquet(str(PROCESSED_PATH) + '/dados_subtipos_limpos.parquet')
print('Loaded rows:', len(df_subtipos))
print(df_subtipos.dtypes)

print("\nDataFrame de Fontes de Recurso:")
df_fontes_recurso = pd.read_parquet(str(PROCESSED_PATH) + '/dados_fontes_recurso_limpos.parquet')
print('Loaded rows:', len(df_fontes_recurso))
print(df_fontes_recurso.dtypes)


Parquet file: E:\VS-Code\Lappis-PS\data\processed

DataFrame de Projetos:
Loaded rows: 800
id_unico                                  string[python]
nome                                      string[python]
cep                                       string[python]
endereco                                  string[python]
descricao                                 string[python]
funcao_social                             string[python]
meta_global                               string[python]
data_inicial_prevista                     datetime64[ns]
data_final_prevista                       datetime64[ns]
data_inicial_efetiva                      datetime64[ns]
data_final_efetiva                        datetime64[ns]
data_cadastro                             datetime64[ns]
especie                                   string[python]
natureza                                  string[python]
natureza_outras                           string[python]
situacao                                  string[pytho

In [7]:
# Configura a conexão com PostgreSQL a partir das variáveis de ambiente
PG_HOST = os.getenv('PG_HOST') or os.getenv('PGHOST')
PG_PORT = os.getenv('PG_PORT') or os.getenv('PGPORT') or '5432'
PG_DB = os.getenv('PG_DB') or os.getenv('PGDATABASE')
PG_USER = os.getenv('PG_USER') or os.getenv('PGUSER')
PG_PASSWORD = os.getenv('PG_PASSWORD') or os.getenv('PGPASSWORD')

if not all([PG_HOST, PG_DB, PG_USER, PG_PASSWORD]):
    raise RuntimeError('Variáveis de conexão PostgreSQL não estão definidas. Configure PG_HOST/PG_DB/PG_USER/PG_PASSWORD no .env')

# cria engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}')
print('Engine created')

# escreve tabela (substitui se já existir)
TABLE_NAME = 'dados_projetos_limpos'
# opção chunksize para evitar uso excessivo de memória em uploads grandes
df_projetos.to_sql(TABLE_NAME, engine, if_exists='replace', index=False, method='multi', chunksize=1000)
print('Table written:', TABLE_NAME)


Engine created


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Validação rápida: conta registros via SQL e mostra primeiras linhas usando pandas.read_sql
with engine.connect() as conn:
    count = conn.execute(f'SELECT COUNT(*) FROM {TABLE_NAME}').scalar()
    print('Row count in DB table:', count)
    sample = pd.read_sql(f'SELECT * FROM {TABLE_NAME} LIMIT 5', conn)
    print(sample.head())
